## Ingestão de dados no Neo4j

Esse jupyter notebook tem o objetivo de fazer a ingestão de dados do arquivo steam.csv que contém dados de jogos vendidos pela plataforma steam

In [121]:
#import libs
from neo4j import GraphDatabase
import pandas as pd

In [122]:
#conexão com o banco de dados
uri = "bolt://localhost:7687"
username = "luiz88"
password = "12345678"

In [123]:
#abre a conexão
driver = GraphDatabase.driver(uri, auth=(username, password))

In [124]:
#leitura do arquivo csv
df = pd.read_csv('steam.csv')

In [125]:
df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [126]:
df.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

In [127]:
#renomeia a coluna name para game para possibilitar o uso do método iloc
df.rename(columns={'name': 'game'}, inplace=True)

In [151]:
#definição dos nós
nodes = ['platforms','developer','required_age','categories',
         'genres','publisher','steamspy_tags']

In [129]:
len(df)

27075

In [130]:
#criação do nó principal games
for line in range(0,len(df)):
    with driver.session() as session:
        session.run(f"""
            CREATE (game:game {{
            appid: '{df.iloc[line].appid}',
            name: '{df.iloc[line].game.replace("'","")}',
            release_date: '{df.iloc[line].release_date}',
            english: '{df.iloc[line].english}',
            achievements: '{df.iloc[line].achievements}',
            positive_ratings: '{df.iloc[line].positive_ratings}',
            negative_ratings: '{df.iloc[line].negative_ratings}',
            average_playtime: '{df.iloc[line].average_playtime}',
            median_playtime: '{df.iloc[line].median_playtime}',
            owners: '{df.iloc[line].owners}',
            price: '{df.iloc[line].price}'
            }})
            """)

In [131]:
#criação dos nós secundários
for n in nodes:
    for item in set(df[n].astype(str).str.split(';').explode().tolist()):
        with driver.session() as session:
            item = item.replace("'","")
            session.run(f"CREATE ({n}:{n} {{name:'{item}'}})")

In [152]:
def create_relationship(game, node_x, x):
    ''''
    Função para criar relacionamento entre o nó game e os demais nós
    '''
    with driver.session() as session:
        x = x.replace("'","")
        game = game.replace("'","")
        session.run(f"""MATCH (a:game {{name: '{game}'}}), 
                        (b:{node_x} {{name: '{x}'}})
                        CREATE (a)-[:BELONGS_TO_{node_x.upper()}]->(b)
                    """)

In [156]:
#criação dos relacionamentos
for node_x in nodes:
    for line in range(0,len(df)):
        game = df.iloc[line].game
        for x in df.astype(str).iloc[line][node_x].split(';'):
            create_relationship(game, node_x, x)